# Gaussian Process Regression

## Import Lib

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm

from torch.xpu import device
from torchvision import models
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

import gpytorch

import matplotlib.pyplot as plt
import numpy as np
import random

In [2]:
# Set a fixed seed value
seed_value = 42
# Set the random seed for Python's built-in random module
random.seed(seed_value)
# Set the random seed for NumPy
np.random.seed(seed_value)
# Set the random seed for PyTorch
torch.manual_seed(seed_value)

# If using CUDA, set the seed for GPU as well (if applicable)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

/home/hskha23/miniconda3/envs/tf/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789122112/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

## Data Preparation

In [4]:
class DatasetGenerator:
    def __init__(self, mnist_data, n_bags=1000, min_instances=3, max_instances=5):
        self.mnist_data = mnist_data
        self.n_bags = n_bags
        self.min_instances = min_instances
        self.max_instances = max_instances
        self.empty_image = torch.zeros(1, 28, 28)  # Create an empty image tensor (1x28x28)

    def create_bags(self):
        bags = []
        labels = []
        
        for _ in range(self.n_bags):
            # Randomly choose a number of instances for the bag
            n_instances = np.random.randint(self.min_instances, self.max_instances + 1)
            
            # Randomly select instances from the dataset
            bag_indices = np.random.choice(len(self.mnist_data), n_instances, replace=False)
            bag_images = [self.mnist_data[i][0] for i in bag_indices]
            
            # Determine the label: 1 if any instance is '9', else 0
            label = 1 if any(self.mnist_data[i][1] == 9 for i in bag_indices) else 0
            
            # Convert images to tensors and pad to ensure exactly 7 instances
            bag_images_tensors = [ToTensor()(img) for img in bag_images]
            while len(bag_images_tensors) < 7:
                bag_images_tensors.append(self.empty_image)  # Pad with empty image
            
            bags.append(torch.stack(bag_images_tensors))
            labels.append(label)

        return bags, labels
    
    def __len__(self):
        return self.n_bags

class TrainDatasetGenerator(DatasetGenerator):
    def __init__(self, mnist_data, n_bags=1000):
        super().__init__(mnist_data, n_bags)

class TestDatasetGenerator(DatasetGenerator):
    def __init__(self, mnist_data, n_bags=500):  # Example: fewer bags for testing
        super().__init__(mnist_data, n_bags)

## Load MNIST dataset

In [5]:
# Load MNIST dataset
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True)

# Create training dataset generator and generate bags
train_generator = TrainDatasetGenerator(mnist_dataset)
train_bags, train_labels = train_generator.create_bags()
train_loader = DataLoader(list(zip(train_bags, train_labels)), batch_size=32, shuffle=True)

# Create test dataset generator and generate bags
test_generator = TestDatasetGenerator(mnist_dataset)
test_bags, test_labels = test_generator.create_bags()
test_loader = DataLoader(list(zip(test_bags, test_labels)), batch_size=16, shuffle=True)

## Define the GP Model with GPyTorch

### PG Likelihood Function

In [6]:
class PGLikelihood(gpytorch.likelihoods._OneDimensionalLikelihood):
    # this method effectively computes the expected log likelihood
    # contribution to Eqn (10) in Reference [1].
    def expected_log_prob(self, target, input, *args, **kwargs):
        mean, variance = input.mean, input.variance
        # Compute the expectation E[f_i^2]
        raw_second_moment = variance + mean.pow(2)

        # Translate targets to be -1, 1
        target = target.to(mean.dtype).mul(2.).sub(1.)

        # We detach the following variable since we do not want
        # to differentiate through the closed-form PG update.
        c = raw_second_moment.detach().sqrt()
        # Compute mean of PG auxiliary variable omega: 0.5 * Expectation[omega]
        # See Eqn (11) and Appendix A2 and A3 in Reference [1] for details.
        half_omega = 0.25 * torch.tanh(0.5 * c) / c

        # Expected log likelihood
        res = 0.5 * target * mean - half_omega * raw_second_moment
        # Sum over data points in mini-batch
        res = res.sum(dim=-1)

        return res

    # define the likelihood
    def forward(self, function_samples):
        return torch.distributions.Bernoulli(logits=function_samples)

    # define the marginal likelihood using Gauss Hermite quadrature
    def marginal(self, function_dist):
        prob_lambda = lambda function_samples: self.forward(function_samples).probs
        probs = self.quadrature(prob_lambda, function_dist)
        return torch.distributions.Bernoulli(probs=probs)

### GP Model

In [7]:
class GPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(0))
        variational_distribution = gpytorch.variational.NaturalVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [8]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

M = 30
inducing_points = torch.linspace(-2., 2., M).unsqueeze(-1).expand(-1, 784)
# inducing_points = torch.randn(M, 1)
model = GPModel(inducing_points)
model.covar_module.base_kernel.lengthscale = 0.2
likelihood = PGLikelihood()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
likelihood = likelihood.to(device)

## Set up optimizer

In [9]:
variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), num_data=train_generator.__len__())

parameters_optimizer = torch.optim.Adam([
    {'params': model.hyperparameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

## Training the model

In [10]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

model.train()
likelihood.train()

mll = gpytorch.mlls.VariationalELBO(likelihood, model, train_labels.__len__())
epochs_iter = tqdm.tqdm(range(100), desc="Epoch")

for i in epochs_iter:
    minibatch_iter = tqdm.tqdm(train_loader, desc="Minibatch", leave=False)
    
    for minibatch, labels in minibatch_iter:
        minibatch, labels = minibatch.to(device), labels.to(device)
        # print(f'Shape of minibatch: {minibatch.shape}')
        minibatch = minibatch.view(-1, 1 * 28 * 28)
        def closure():
            parameters_optimizer.zero_grad()
            output = model(minibatch)
            loss = -mll(output, labels)
            return loss
        
        parameters_optimizer.step(closure)
        
        def closure():
            variational_ngd_optimizer.zero_grad()
            output = model(minibatch)
            loss = -mll(output, labels)
            return loss
        
        variational_ngd_optimizer.step(closure)

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]   


RuntimeError: The size of tensor a (32) must match the size of tensor b (224) at non-singleton dimension 0

## Testing

In [ ]:
model.eval()
likelihood.eval()

with torch.inference_mode():
    correct = 0
    total = 0
    for minibatch, labels in test_loader:
        minibatch, labels = minibatch.to(device), labels.to(device)
        
        with gpytorch.settings.max_preconditioner_size(10), torch.no_grad():
            output = model(minibatch)
            preds = likelihood(output).probs.mean > 0.5
            correct += preds.eq(labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    print(f"Accuracy: {accuracy}")

## References